# Interpretable Machine Learning for Credit Risk Assessment using SHAP and LIME


In [9]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=9715dfef93173603a1d76d3dd416f160409dc4160748d09f25bf492eafdd79fa
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


In [11]:
import os
import warnings
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from xgboost import XGBClassifier

import shap
from lime.lime_tabular import LimeTabularExplainer

warnings.filterwarnings("ignore")
plt.rcParams.update({"figure.max_open_warning": 0})

# -----------------------------
# Configuration / file paths
# -----------------------------
DATA_PATH = "/content/credit_risk_dataset.csv"   # <= uses your CSV
OUTPUT_DIR = "/mnt/data/credit_risk_project_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

MODEL_PATH = os.path.join(OUTPUT_DIR, "xgb_model.joblib")
SCALER_PATH = os.path.join(OUTPUT_DIR, "scaler.joblib")
TEST_PRED_CSV = os.path.join(OUTPUT_DIR, "test_predictions.csv")

SHAP_GLOBAL_PNG = os.path.join(OUTPUT_DIR, "shap_global_summary.png")
SHAP_LOCAL_PREFIX = os.path.join(OUTPUT_DIR, "shap_local_")   # will append names
LIME_HTML_PREFIX = os.path.join(OUTPUT_DIR, "lime_local_")    # will append names + .html

In [13]:
# -----------------------------
# 1) Load dataset
# -----------------------------
print("Loading dataset from:", DATA_PATH)
df = pd.read_csv("/content/credit_risk_dataset.csv")
print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())
print()

Loading dataset from: /content/credit_risk_dataset.csv
Dataset shape: (3000, 10)
Columns: ['loan_amount', 'term', 'interest_rate', 'annual_income', 'credit_score', 'dti', 'delinquencies_2yrs', 'revol_util', 'employment_length', 'default']



In [14]:
# -----------------------------
# 2) Basic checks & split
# -----------------------------
# Expect 'default' column as target
if "default" not in df.columns:
    raise ValueError("Dataset must contain a 'default' column as target (0/1).")


In [15]:
# Features (all except target)
X_raw = df.drop(columns=["default"]).copy()
y = df["default"].copy()


In [16]:
# Keep feature names
feature_names = X_raw.columns.tolist()


In [17]:
# Train/test split (stratify to preserve class ratio)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.25, random_state=42, stratify=y
)

print("Train shape:", X_train_raw.shape, "Test shape:", X_test_raw.shape)
print("Positive class ratio (train):", y_train.mean(), " (test):", y_test.mean())
print()


Train shape: (2250, 9) Test shape: (750, 9)
Positive class ratio (train): 0.31955555555555554  (test): 0.31866666666666665



In [18]:
# -----------------------------
# 3) Preprocessing (Scaling)
# -----------------------------
# We'll scale numeric features. If you have categorical variables in the CSV,
# you'd normally encode them; this synthetic dataset is numeric-friendly.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)


In [19]:
# Save scaler for later reuse
joblib.dump(scaler, SCALER_PATH)


['/mnt/data/credit_risk_project_outputs/scaler.joblib']

In [20]:
# -----------------------------
# 4) Train XGBoost classifier
# -----------------------------
print("Training XGBoost classifier...")
model = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42,
    verbosity=0
)

model.fit(X_train, y_train)

Training XGBoost classifier...


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [21]:
# Save model
joblib.dump(model, MODEL_PATH)
print("Model saved to:", MODEL_PATH)
print()

Model saved to: /mnt/data/credit_risk_project_outputs/xgb_model.joblib



In [22]:
# -----------------------------
# 5) Evaluate on test set
# -----------------------------
pred_proba = model.predict_proba(X_test)[:, 1]
pred_label = (pred_proba >= 0.5).astype(int)

auc = roc_auc_score(y_test, pred_proba)
f1 = f1_score(y_test, pred_label)

print("Test AUC: {:.4f}".format(auc))
print("Test F1: {:.4f}".format(f1))
print("Classification report:")
print(classification_report(y_test, pred_label))


Test AUC: 0.5434
Test F1: 0.1953
Classification report:
              precision    recall  f1-score   support

           0       0.68      0.87      0.77       511
           1       0.33      0.14      0.20       239

    accuracy                           0.64       750
   macro avg       0.51      0.50      0.48       750
weighted avg       0.57      0.64      0.58       750



In [23]:
# Save test predictions (attach original unscaled features for readability)
results_df = X_test_raw.copy().reset_index(drop=True)
results_df["true_default"] = y_test.reset_index(drop=True)
results_df["pred_proba_default"] = pred_proba
results_df["pred_label_default"] = pred_label
results_df.to_csv(TEST_PRED_CSV, index=False)
print("Test predictions saved to:", TEST_PRED_CSV)
print()

Test predictions saved to: /mnt/data/credit_risk_project_outputs/test_predictions.csv



In [24]:
# -----------------------------
# 6) SHAP: global explanation
# -----------------------------
print("Computing SHAP values (global)... This may take a bit.")
explainer = shap.TreeExplainer(model, feature_perturbation="tree_path_dependent")

Computing SHAP values (global)... This may take a bit.


In [25]:
# compute shap values for a sample (pass training data)
shap_values_train = explainer.shap_values(X_train)  # shape (n_samples, n_features) for binary


In [26]:
# SHAP summary plot
# Use the original unscaled feature names and the *unscaled* training dataframe to make the plot readable.
# We can pass the scaled data and feature names, but to show real units it's nicer to pass unscaled values.
# We'll plot using the scaled data but provide feature names.
shap.summary_plot(shap_values_train, X_train, feature_names=feature_names, show=False)
plt.title("SHAP Global Summary (train)")
plt.tight_layout()
plt.savefig(SHAP_GLOBAL_PNG, dpi=300, bbox_inches="tight")
plt.close()
print("Saved SHAP global summary to:", SHAP_GLOBAL_PNG)
print()

Saved SHAP global summary to: /mnt/data/credit_risk_project_outputs/shap_global_summary.png



In [27]:
# -----------------------------
# 7) SHAP: local explanations (3 cases)
# -----------------------------
# We'll select:
#  - clear_approval: predicted prob <= 0.1 (and true label 0 if available)
#  - clear_denial: predicted prob >= 0.9 (and true label 1 if available)
#  - borderline: predicted prob between 0.45 and 0.55 if possible

test_proba_series = pd.Series(pred_proba, index=range(len(pred_proba)))
test_true_series = y_test.reset_index(drop=True)

def find_case(mask):
    idxs = test_proba_series[mask].index.tolist()
    return idxs[0] if idxs else None

idx_clear_approval = find_case(test_proba_series <= 0.1)
idx_clear_denial = find_case(test_proba_series >= 0.9)
idx_borderline = find_case((test_proba_series > 0.45) & (test_proba_series < 0.55))


In [28]:
# Fallback logic if any case not found
if idx_clear_approval is None:
    idx_clear_approval = test_proba_series.idxmin()  # best available
if idx_clear_denial is None:
    idx_clear_denial = test_proba_series.idxmax()
if idx_borderline is None:
    # pick closest to 0.5
    idx_borderline = (test_proba_series - 0.5).abs().idxmin()

cases = {
    "clear_approval": idx_clear_approval,
    "clear_denial": idx_clear_denial,
    "borderline": idx_borderline
}

print("Selected case indices (in test set):", cases)
print()

Selected case indices (in test set): {'clear_approval': 3, 'clear_denial': 343, 'borderline': 8}



In [29]:
# Generate SHAP force plots (matplotlib) and waterfall plots for each selected case
for name, idx in cases.items():
    x_scaled = X_test[idx].reshape(1, -1)
    x_raw = X_test_raw.reset_index(drop=True).loc[idx:idx]  # dataframe slice for human-readable values


In [30]:
# shap values for the instance
sv = explainer.shap_values(x_scaled)  # shape (1, n_features)


In [31]:
# Force plot (matplotlib)
# shap.force_plot supports matplotlib with shap values + base value + feature display
try:
    # supply a 1D array of shap values
    shap_fig = shap.force_plot(
        explainer.expected_value,
        sv,
        matplotlib=True,
        feature_names=feature_names,
        show=False
    )
    # shap.force_plot returns a matplotlib.figure.Figure when matplotlib=True
    plt.tight_layout()
    out_path = SHAP_LOCAL_PREFIX + f"{name}.png"
    plt.savefig(out_path, bbox_inches="tight", dpi=300)
    plt.close()
    print(f"Saved SHAP local (force) plot for {name} -> {out_path}")
except Exception as e:
    # Fallback: produce a bar chart of absolute shap contributions
    abs_sv = np.abs(sv).reshape(-1)
    order = np.argsort(-abs_sv)[:10]  # top 10
    top_feats = [feature_names[i] for i in order]
    top_vals = sv.reshape(-1)[order]

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.barh(top_feats[::-1], top_vals[::-1])
    ax.set_title(f"SHAP local (approx) {name}")
    plt.tight_layout()
    out_path = SHAP_LOCAL_PREFIX + f"{name}_fallback.png"
    plt.savefig(out_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"Fallback SHAP plot saved for {name} -> {out_path} (error: {e})")
    print()

Saved SHAP local (force) plot for borderline -> /mnt/data/credit_risk_project_outputs/shap_local_borderline.png


In [32]:
# -----------------------------
# 8) LIME: local explanations
# -----------------------------
# Lime expects a predict_fn that accepts raw (unscaled) data and returns probabilities.
def predict_fn_unscaled(raw_array_np):
    """
    raw_array_np: 2D numpy array in original feature space (unscaled)
    returns: probability for class 1 (default) as shape (n_samples, 2) like predict_proba
    """
    # scale then pass to model
    scaled = scaler.transform(raw_array_np)
    return model.predict_proba(scaled)


In [33]:

# -----------------------------
# 8) LIME: local explanations
# -----------------------------
# Lime expects a predict_fn that accepts raw (unscaled) data and returns probabilities.
def predict_fn_unscaled(raw_array_np):
    """
    raw_array_np: 2D numpy array in original feature space (unscaled)
    returns: probability for class 1 (default) as shape (n_samples, 2) like predict_proba
    """
    # scale then pass to model
    scaled = scaler.transform(raw_array_np)
    return model.predict_proba(scaled)

# Create LIME explainer using training raw data (unscaled) for interpretable feature distribution
lime_explainer = LimeTabularExplainer(
    training_data=np.array(X_train_raw),
    feature_names=feature_names,
    class_names=["No Default", "Default"],
    mode="classification",
    random_state=42
)

for name, idx in cases.items():
    # get the original raw instance (1D array)
    instance_raw = X_test_raw.reset_index(drop=True).loc[idx].values
    exp = lime_explainer.explain_instance(
        instance_raw,
        predict_fn_unscaled,
        num_features=min(len(feature_names), 8)
    )
    out_html = LIME_HTML_PREFIX + f"{name}.html"
    exp.save_to_file(out_html)
    print(f"Saved LIME explanation for {name} -> {out_html}")

print()

Saved LIME explanation for clear_approval -> /mnt/data/credit_risk_project_outputs/lime_local_clear_approval.html
Saved LIME explanation for clear_denial -> /mnt/data/credit_risk_project_outputs/lime_local_clear_denial.html
Saved LIME explanation for borderline -> /mnt/data/credit_risk_project_outputs/lime_local_borderline.html



In [35]:

# -----------------------------
# 9) Summaries / small textual outputs saved
# -----------------------------
summary_txt = os.path.join(OUTPUT_DIR, "report_summary.txt")
with open(summary_txt, "w") as f:
    f.write("Interpretable Credit Risk Project - Summary\n")
    f.write("=========================================\n\n")
    f.write(f"Data path: {DATA_PATH}\n")
    f.write(f"Output dir: {OUTPUT_DIR}\n\n")
    f.write("Model performance on test set:\n")
    f.write(f" - AUC: {auc:.4f}\n")
    f.write(f" - F1: {f1:.4f}\n\n")
    f.write("Selected local cases (test indices):\n")
    for name, idx in cases.items():
        f.write(f" - {name}: test_index = {idx}, pred_proba = {test_proba_series[idx]:.4f}, true = {test_true_series[idx]}\n")

print("Summary report saved to:", summary_txt)
print()
print("All done. Outputs written to:", OUTPUT_DIR)
print("Files of interest:")
for fname in sorted(os.listdir(OUTPUT_DIR)):
    print(" -", fname)

Summary report saved to: /mnt/data/credit_risk_project_outputs/report_summary.txt

All done. Outputs written to: /mnt/data/credit_risk_project_outputs
Files of interest:
 - lime_local_borderline.html
 - lime_local_clear_approval.html
 - lime_local_clear_denial.html
 - report_summary.txt
 - scaler.joblib
 - shap_global_summary.png
 - shap_local_borderline.png
 - test_predictions.csv
 - xgb_model.joblib
